In [ ]:
import cv2
import glob
import time
import numpy as np
import IPython.display
from PIL import Image
from deepface import DeepFace
from scipy.spatial.distance import cosine

In [ ]:
target_size = (224, 224)
my_images = glob.glob("my_images2/*")
threshold = 0.85
embeddings_dict = {}

In [ ]:
def my_embeddings():
    image_features = []

    for img_path in my_images:
        try:
            embedding = DeepFace.represent(img_path, model_name="Facenet", enforce_detection=False, align=False)
            embedding = embedding[0]["embedding"]
            embeddings_dict[img_path] = embedding
            image_features.append(embedding)
        except:
            print(f"No face detected in {img_path}")

    image_features = np.array(image_features)
    return image_features

embeddings = my_embeddings()
np.save("face_embeddings.npy", embeddings_dict)

In [ ]:
def image_embedding(img):
    try:
        similarities = []
        embedding = DeepFace.represent(img, model_name="Facenet", enforce_detection=False, align=False)
        embedding = embedding[0]
        face = embedding["facial_area"]
        x, y, w, h = face["x"], face["y"], face["w"], face["h"]
        for em in embeddings:
            similarity = 1 - cosine(em, embedding["embedding"])
            similarities.append(similarity)
        max_similarity = max(similarities)
        print(max_similarity)
        return embedding["embedding"], True, (x, y, w, h), max_similarity
    except:
        print("No face detected!")
        return None, False, None, 0

In [ ]:
test_img = cv2.imread("sample.jpg")
embedding, result, face_info, max_similarity = image_embedding(test_img)
print("This is you, Abolfazl!") if max_similarity > threshold else print("This is not you.")

In [ ]:
def frame_show(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(rgb_frame)
    IPython.display.display(img)

In [ ]:
cam = cv2.VideoCapture(0)
cv2.namedWindow("Abolfazl Detector")
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

frame_counter = 0
skip_threshold = 20
display_duration = 1.5

last_detection_time = 0
last_result = None
last_face_info = None
last_max_similarity = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame_counter += 1

    if frame_counter % skip_threshold == 0:
        embedding, result, face_info, max_similarity = image_embedding(frame)

        if embedding is not None:
            last_detection_time = time.time()
            last_result = result
            last_face_info = face_info
            last_max_similarity = max_similarity

    if last_result is not None and time.time() - last_detection_time < display_duration:
        x, y, w, h = last_face_info
        sm = f"%{max_similarity * 100}"[:6]
        text = "This is not you."

        (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
        (sm_width, sm_height), _ = cv2.getTextSize(sm, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)

        x_text = x
        y_text = y - 10
        x_sm = x_text + (text_width - sm_width) // 2
        y_sm = y_text - 25

        color = (0, 0, 255)

        if last_max_similarity > threshold:
            text = "This is you, Abolfazl!"
            color = (0, 255, 0)

        if last_result:
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, text, (x_text, y_text), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.8, color, 2, cv2.LINE_AA)
            cv2.putText(frame, sm, (x_sm, y_sm), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.8, color, 2, cv2.LINE_AA)

    cv2.imshow("Abolfazl Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()